In [68]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from newspaper import Article
import string
import re

ARTICLE_LIMIT = 2000
LEFT_LEANING = 0
RIGHT_LEANING = 1

def get_number_rating(vote):
    switcher = {
    "Positive": 1,
    "SomewhatPositive": 0.5,
    "Neutral": 0,
    "SomewhatNegative": -0.5,
    "Negative": -1
    }
    return switcher.get(vote, "N/A")

def clean(article):
    cleaned_article = re.sub('[\n\t,]', '', article)
    return cleaned_article

def get_text(url):
    article = Article(url)
    article.download()
    article.parse()
    article_text = article.text
    if article_text == '':
        raise Exception("Could not locate article body")
    cleaned_article_text = clean(article_text)
    return cleaned_article_text

def add_to_df(article, bias, dataframe):
    article_and_bias = [article, bias]
    row = pd.Series(article_and_bias, index=dataframe.columns)
    new_dataframe = dataframe.append(row, ignore_index=True)
    return new_dataframe

all_articles = pd.read_csv('newsArticlesWithLabels.tsv', sep='\t')

democrat_ratings = all_articles.loc[:ARTICLE_LIMIT, 'democrat.vote']
republican_ratings = all_articles.loc[:ARTICLE_LIMIT, 'republican.vote']
urls = all_articles.loc[:ARTICLE_LIMIT, 'url']

errors = []
article_bias = pd.DataFrame(columns=['article', 'bias'])

for i in range(len(urls)):
    try:
        diff = get_number_rating(democrat_ratings[i])-get_number_rating(republican_ratings[i])
        if  diff > 0:
            article_bias = add_to_df(get_text(urls[i]), LEFT_LEANING, article_bias)
            print(f"SUCCESS ({i})")
        elif diff < 0:
            article_bias = add_to_df(get_text(urls[i]), RIGHT_LEANING, article_bias)
            print(f"SUCCESS ({i})")
    except Exception as e: 
        print(f"FAILURE ({i})")
        errors.append(e)

article_bias.to_csv("binary_bias.csv")
article_bias

FAILURE (6)
SUCCESS (12)
SUCCESS (17)
SUCCESS (18)
SUCCESS (20)
SUCCESS (21)
FAILURE (23)
SUCCESS (24)
FAILURE (25)
SUCCESS (26)
SUCCESS (27)
FAILURE (29)
FAILURE (30)
FAILURE (31)
FAILURE (32)
FAILURE (33)
FAILURE (34)
SUCCESS (36)
FAILURE (37)
SUCCESS (38)
SUCCESS (39)
FAILURE (40)
SUCCESS (42)
FAILURE (43)
FAILURE (44)
FAILURE (45)
SUCCESS (47)
SUCCESS (48)
SUCCESS (50)
FAILURE (51)
SUCCESS (52)
SUCCESS (53)
SUCCESS (55)
SUCCESS (56)
SUCCESS (57)
SUCCESS (60)
SUCCESS (62)
SUCCESS (64)
FAILURE (65)
FAILURE (69)
SUCCESS (72)
SUCCESS (73)
SUCCESS (75)
FAILURE (77)
FAILURE (78)
FAILURE (79)
SUCCESS (82)
FAILURE (83)
FAILURE (87)
FAILURE (89)
SUCCESS (90)
SUCCESS (92)
SUCCESS (96)
SUCCESS (97)
SUCCESS (99)
SUCCESS (102)
FAILURE (103)
FAILURE (104)
FAILURE (108)
SUCCESS (111)
SUCCESS (113)
FAILURE (116)
SUCCESS (117)
FAILURE (118)
SUCCESS (119)
SUCCESS (120)
FAILURE (121)
SUCCESS (123)
SUCCESS (124)
SUCCESS (127)
SUCCESS (130)
SUCCESS (133)
SUCCESS (139)
SUCCESS (142)
FAILURE (143)
SUCCES

D:\Anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 433 bytes but only got 0. Skipping tag 33432
  warnings.warn(
D:\Anaconda\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


SUCCESS (831)
SUCCESS (832)
FAILURE (833)
SUCCESS (834)
SUCCESS (835)
SUCCESS (837)
SUCCESS (838)
SUCCESS (841)
FAILURE (842)
SUCCESS (844)
FAILURE (846)
FAILURE (847)
FAILURE (848)
SUCCESS (849)
SUCCESS (850)
SUCCESS (851)
SUCCESS (852)
SUCCESS (853)
SUCCESS (854)
SUCCESS (855)
SUCCESS (856)
SUCCESS (857)
SUCCESS (858)
SUCCESS (859)
FAILURE (860)
SUCCESS (861)
SUCCESS (862)
SUCCESS (864)
SUCCESS (865)
SUCCESS (866)
FAILURE (867)
SUCCESS (868)
SUCCESS (869)
FAILURE (870)
SUCCESS (871)
SUCCESS (872)
SUCCESS (874)
SUCCESS (875)
SUCCESS (876)
FAILURE (877)
SUCCESS (878)
SUCCESS (880)
SUCCESS (881)
SUCCESS (882)
FAILURE (883)
SUCCESS (884)
SUCCESS (885)
SUCCESS (887)
SUCCESS (888)
SUCCESS (890)
SUCCESS (891)
SUCCESS (892)
SUCCESS (893)
SUCCESS (894)
SUCCESS (895)
SUCCESS (897)
SUCCESS (898)
SUCCESS (899)
SUCCESS (901)
SUCCESS (902)
SUCCESS (903)
SUCCESS (904)
FAILURE (906)
SUCCESS (907)
SUCCESS (908)
SUCCESS (909)
SUCCESS (910)
SUCCESS (912)
SUCCESS (915)
SUCCESS (917)
SUCCESS (918)
FAILUR

D:\Anaconda\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


SUCCESS (1214)
SUCCESS (1216)
SUCCESS (1217)
SUCCESS (1218)
SUCCESS (1219)
SUCCESS (1220)
SUCCESS (1221)
FAILURE (1224)
SUCCESS (1225)
SUCCESS (1226)
SUCCESS (1227)
SUCCESS (1231)
SUCCESS (1232)
SUCCESS (1233)
SUCCESS (1237)
SUCCESS (1238)
SUCCESS (1240)
SUCCESS (1241)
SUCCESS (1242)
SUCCESS (1255)
SUCCESS (1256)
SUCCESS (1257)
FAILURE (1258)
SUCCESS (1259)
SUCCESS (1260)
FAILURE (1261)
SUCCESS (1265)
SUCCESS (1269)
FAILURE (1273)
FAILURE (1274)
SUCCESS (1276)
SUCCESS (1286)
SUCCESS (1292)
SUCCESS (1300)
FAILURE (1303)
SUCCESS (1304)
SUCCESS (1305)
SUCCESS (1307)
SUCCESS (1314)
SUCCESS (1315)
SUCCESS (1320)
SUCCESS (1327)
SUCCESS (1328)
SUCCESS (1333)
SUCCESS (1337)
SUCCESS (1347)
SUCCESS (1348)
SUCCESS (1349)
SUCCESS (1352)
FAILURE (1355)
FAILURE (1356)
SUCCESS (1358)
SUCCESS (1366)
SUCCESS (1368)
SUCCESS (1370)
SUCCESS (1371)
SUCCESS (1372)
FAILURE (1374)
SUCCESS (1376)
FAILURE (1378)
SUCCESS (1379)
FAILURE (1380)
SUCCESS (1381)
SUCCESS (1382)
FAILURE (1383)
SUCCESS (1385)
FAILURE (1

D:\Anaconda\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 2. 
  warnings.warn(str(msg))


SUCCESS (1676)
SUCCESS (1680)
SUCCESS (1683)
SUCCESS (1684)
SUCCESS (1685)
SUCCESS (1688)
SUCCESS (1693)
SUCCESS (1694)
SUCCESS (1696)
FAILURE (1697)
SUCCESS (1698)
SUCCESS (1699)
FAILURE (1700)
SUCCESS (1701)
SUCCESS (1702)
SUCCESS (1704)
SUCCESS (1705)
SUCCESS (1706)
SUCCESS (1707)
SUCCESS (1709)
SUCCESS (1711)
SUCCESS (1712)
SUCCESS (1713)
FAILURE (1717)
SUCCESS (1718)
FAILURE (1719)
SUCCESS (1720)
SUCCESS (1723)
SUCCESS (1724)
SUCCESS (1725)
FAILURE (1727)
SUCCESS (1733)
SUCCESS (1735)
SUCCESS (1739)
SUCCESS (1740)
SUCCESS (1741)
SUCCESS (1742)
FAILURE (1743)
SUCCESS (1746)
SUCCESS (1747)
SUCCESS (1750)
SUCCESS (1752)
SUCCESS (1753)
SUCCESS (1754)
SUCCESS (1759)
SUCCESS (1760)
SUCCESS (1762)
SUCCESS (1767)
SUCCESS (1768)
SUCCESS (1771)
FAILURE (1772)
SUCCESS (1776)
SUCCESS (1778)
SUCCESS (1779)
SUCCESS (1785)
FAILURE (1786)
SUCCESS (1787)
SUCCESS (1790)
SUCCESS (1795)
SUCCESS (1796)
SUCCESS (1799)
SUCCESS (1801)
FAILURE (1802)
SUCCESS (1807)
FAILURE (1809)
FAILURE (1811)
SUCCESS (1

,article,bias
0,The following irresponsible police action repo...,0
1,SACRAMENTO — “Living in parallel universes” is...,1
2,"Co-host of MSNBC's ""The Cycle"" Touré joined Hu...",1
3,US Representative Michele Bachmann speaks duri...,1
4,The agency missed a Feb. 15 deadline to comple...,1
...,...,...
818,Olga Rudenko and Jesse SingalSpecial to USA TO...,0
819,Susan Page USA TODAYAmericans overwhelmingly s...,1
820,Story highlights Top diplomatic economic offic...,0
821,next Image 1 of 2prev Image 2 of 2A Republican...,1


In [79]:
binary_bias = pd.read_csv('binary_bias.csv')
binary_bias


,article,bias
0,The following irresponsible police action repo...,0
1,SACRAMENTO — “Living in parallel universes” is...,1
2,"Co-host of MSNBC's ""The Cycle"" Touré joined Hu...",1
3,US Representative Michele Bachmann speaks duri...,1
4,The agency missed a Feb. 15 deadline to comple...,1
...,...,...
818,Olga Rudenko and Jesse SingalSpecial to USA TO...,0
819,Susan Page USA TODAYAmericans overwhelmingly s...,1
820,Story highlights Top diplomatic economic offic...,0
821,next Image 1 of 2prev Image 2 of 2A Republican...,1


In [80]:
len(article_bias)

823

In [19]:
from newspaper import Config

url = "http://online.wsj.com/news/articles/SB10001424052702304527504579169853503880212"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.75'
config = Config()
config.browser_user_agent = user_agent

test = Article(url.strip())
test.download()
test.parse()
test.text

ArticleException: Article `download()` failed with 404 Client Error: Not Found for url: http://online.wsj.com/news/articles/SB10001424052702304527504579169853503880212 on URL http://online.wsj.com/news/articles/SB10001424052702304527504579169853503880212

In [34]:
url = "http://www.nytimes.com/2013/03/07/us/politics/cias-harsh-interrogations-pose-hurdles-for-john-brennan.html"

article = Article(url)
article.download()
article.parse()
article_text = article.text
cleaned_article_text = clean(article_text)
cleaned_article_text

'The agency missed a Feb. 15 deadline to complete a review of the report which has 35000 footnotes referring to 6 million documents from C.I.A. files. It now appears likely that the response offering the committee any factual corrections or broader judgments will be delayed until Mr. Brennan’s arrival.Because Mr. Obama famously said he preferred to look forward not back at his predecessor’s counterterrorism programs the Senate report is by far the most thorough examination of how the United States came to use nudity cold sleep deprivation stress positions wall-slamming and waterboarding methods it had long condemned as abuse or torture.Mr. Brennan will have to decide whether to support making a redacted version of the interrogation report public as the committee is likely to support after the C.I.A. completes its review and as a United Nations human rights adviser urged this week. Several Democratic senators and at least one Republican Senator John McCain of Arizona who was tortured as

In [36]:

URLs = []
for column in all_articles:
    URLs.append(all_articles[column][23])
URLs

['http://video.foxnews.com/v/2800623370001/are-media-casting-obama-as-disengaged/',
 'Opinion',
 0,
 'Democrat Scandals',
 'Civil Rights',
 'Negative',
 'SomewhatNegative']